<a href="https://colab.research.google.com/github/FarazNadeem22/Data-Strcutures/blob/master/MexicoPension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, Any
import time

# Simulation parameters

In [125]:
# Simulation parameters
NUM_SIMULATIONS = 1_0_0
BUDGET: int = 8_500_000_000
COST_PER_PERSON: int = 500 * 12
MAX_COVERAGE: int = BUDGET // COST_PER_PERSON
TOTAL_POOR = 1_872_313

# Enrollment data with leakage and undercoverage rates
OPTIONS: Dict[str, Dict[str, Any]] = {
    "Option 1": {
        "Poor": 637_897,
        "Not_poor": 221_402,
        "Undercoverage": .2576,
        "Leakage": .6593
         },
    "Option 2": {
        "Poor": 801_537,
        "Not_poor": 611_645,
        "Undercoverage": 0.4328,
        "Leakage": .5719
        },
    "Option 3": {
        "Poor": 1_047_934,
        "Not_poor": 1_992_910,
        "Undercoverage": .6554,
        "Leakage": .4403
        }
    }

# Monte Carlo Simulation

In [126]:
# Monete Carlo simulation function
def monte_carlo_simulation(num_simulations: int = NUM_SIMULATIONS, max_coverage: int = MAX_COVERAGE, options: Dict[str, Any] = OPTIONS) -> pd.DataFrame:
  """
  Runs a Monte Carlo simulation to estimate the expected number of poor and not-poor individuals covered under budget
  constraints for a given policy option.

  Returns:
  pd.DataFrame: DataFrame containing the simulated coverage results.
  """
  results = []

  for _ in range(num_simulations):
    total_enrolled : int = options['Poor'] + options['Not_poor']

    # Leakage
    not_poor_covered: int = np.random.binomial(options['Not_poor'], options['Leakage'])

    # Reamining budget
    remaining_budget: int = MAX_COVERAGE - (not_poor_covered)

    # Cover as many poor people as possible with the remaining budget
    poor_covered: int = min(remaining_budget, options['Poor'])

    # Check if we still have the budget to cover more poor people
    uncovered_poor: int = options['Poor'] - poor_covered

    # Check for budget left
    budget_left_over = remaining_budget - poor_covered

    # Ensure that budget left over is more than 0 and poor people are more than 0
    if budget_left_over > 0 and uncovered_poor > 0:
        additional_coverage: int = min(uncovered_poor, budget_left_over)
        poor_covered += additional_coverage

    total_people_covered: int = poor_covered + not_poor_covered

    results.append({
        "Poor_covered": poor_covered,
        "Not_poor_covered": not_poor_covered,
        "Total_covered": total_people_covered,
      }
    )

  return pd.DataFrame(results)


In [127]:
def main() -> None:
    """
    Main function to execute Monte Carlo simulations for all policy options and display summary statistics.
    """
    # Keeping track of how long the function takes to execute
    t1 = time.time()

    # Run simulation
    simulation_results: Dict[str, pd.DataFrame] = {option: monte_carlo_simulation(options=data) for option, data in OPTIONS.items()}

    # Summary Stats in simulation_results
    summary_stats: Dict[str, pd.DataFrame] = {}
    for option, df in simulation_results.items():
        stats = df.describe().apply(lambda s: s.map('{0:,.2f}'.format) if s.dtype == "float64" else s)
        summary_stats[option] = stats

    # Display summary stats for each option
    for option, stats in summary_stats.items():
        print(f"\nSummary statistics for {option}:")
        print(stats)
        # Get Leagake
        mean_not_poor_covered = float(stats.loc['mean', 'Not_poor_covered'].replace(',', ''))
        leakage_percentage = (mean_not_poor_covered / TOTAL_POOR) * 100
        print(f"\nLeakage: {mean_not_poor_covered}/{TOTAL_POOR} = {leakage_percentage:.2f}%")

    # Display time taken
    print(f"Time taken: {time.time() - t1:.4f} seconds")

# Ensure OPTIONS is defined before running main()
if __name__ == "__main__":
    main()



Summary statistics for Option 1:
      Poor_covered Not_poor_covered Total_covered
count       100.00           100.00        100.00
mean    637,897.00       145,952.76    783,849.76
std           0.00           194.36        194.36
min     637,897.00       145,434.00    783,331.00
25%     637,897.00       145,818.50    783,715.50
50%     637,897.00       145,951.50    783,848.50
75%     637,897.00       146,083.50    783,980.50
max     637,897.00       146,492.00    784,389.00

Leakage: 145952.76/1872313 = 7.80%

Summary statistics for Option 2:
      Poor_covered Not_poor_covered Total_covered
count       100.00           100.00        100.00
mean    801,537.00       349,840.08  1,151,377.08
std           0.00           391.02        391.02
min     801,537.00       348,671.00  1,150,208.00
25%     801,537.00       349,590.75  1,151,127.75
50%     801,537.00       349,829.50  1,151,366.50
75%     801,537.00       350,127.25  1,151,664.25
max     801,537.00       351,023.00  1,152,560